# Neural Networks for Data Science Applications
## Homework 1: Saliency maps for interpretability

**Name**: Ramona Tarantino

**Matricola**: 2082006

> ✍ Upload the completed notebook **before 10/11/2023 at 23:59** on the Google Classroom page.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# To ensure reproducible results (as much as possible)
tf.keras.utils.set_random_seed(1234)

### Overview

Neural networks are powerful tools, but they are **black-boxes**, meaning that it is difficult to provide human-understandable explanations on what they are doing. The field of **explanaibility** is concerned with finding algorithms for achieving this. In this homework, you will be guided in implementing some basic explanaibility algorithms (**saliency maps**), which is an instructive way of playing with the TensorFlow autodiff framework.

### Instructions

1. The homework is divided into four mandatory exercises (**5 points in total**), and a few optional exercises. Optional exercises are provided if you like the topic and would like to explore more; you are free to ignore them or complete as many as you want. I will not grade them but I might provide feedback for especially nice solutions.
2. Completing the homework successfully will remove 1 exercise from the end-of-term homework.
3. If your grade does not satisfy you, you are also free to complete the full EoT homework to recover it.
3. The grade can be kept for the entire academic year (up to October 2024).

**IMPORTANT - read carefully before starting**:

> 🟨 *External material*: if you use any external material or inspiration for the code, reference it *explicitly* in the corresponding cell. For the textual descriptions, copy-paste *is not allowed*. <ins>Not following these two points is an immediate 0 mark</ins>.

> 🔵 *Grammar*: for the textual descriptions, I will remove points for too many grammatical or textual errors. Please try to be precise and provide nice-to-read descriptions, like if you were writing a report.

> 🟥 *Vectorization and TensorFlow*: the homework must be done _fully in TensorFlow_ and vectorizing the code as much as possible (e.g., do not loop explicitly over the batch dimension).

> 🟪 *Math*: you can also use LaTeX in Markdown if you need to write equations or if you need generic math notation.

## Warmup: Data loading

For this homework, you can select any **tabular dataset** that you like, for either classification or regression. A few repositories that you can look at:

1. The catalog of [TensorFlow Datasets](https://www.tensorflow.org/datasets/).
2. The [Kaggle catalog](https://www.kaggle.com/data). For downloading data from Kaggle on Google Colab, you will need to [load your Kaggle authentication token](https://colab.research.google.com/github/corrieann/kaggle/blob/master/kaggle_api_in_colab.ipynb).
3. The [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php).
4. The [🤗 HuggingFace Datasets](https://huggingface.co/docs/datasets/) repository.

You are not bound to these; any open repository is okay. The choice of dataset will not influence the mark.

✍ **DESCRIPTION OF THE CODE**

*Provide a small description of the dataset below (e.g., source, task, bibliographic reference if necessary...), both as text and in the comments of the code.*

**TODO**: add description here (1-2 paragraphs).

The Iris dataset is a multivariate dataset introduced by the British statistician and biologist Ronald Fisher in his 1936 paper. The dataset consists of 150 samples from three species of Iris (Iris setosa, Iris virginica, and Iris versicolor) stored in a 150x4 numpy.ndarray. Four features were measured from each sample: the length and the width of the sepals and petals, in centimeters. The rows being the samples and the columns being: Sepal Length, Sepal Width, Petal Length and Petal Width.



The task associated with this dataset is to classify the samples into one of the three Iris species based on the attribute measurements. This dataset is widely used for educational purposes in the fields of machine learning and statistics due to its simplicity and the fact that it is a well-understood problem. The dataset is freely available in the UCI Machine Learning Repository, and it is also included with the scikit-learn machine learning library for Python.


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf

# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the feature data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert labels to one-hot encoding for neural network
y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test_one_hot = tf.keras.utils.to_categorical(y_test, num_classes=3)



### Exercise 1: Train a neural network model (1 point)

Define, train, and test a neural network for the provided dataset.

☝ You are free to make any modelling choice (e.g., activation function, normalization layers, etc.), provided the result makes sense.

✅ **Completion requirement**: print on screen the test accuracy of the network. Additional comments and visualizations are also appreciated.

In [ ]:
# TODO: define a suitable neural network.

model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # Output layer for 3 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 40        
                                                                 
 dense_4 (Dense)             (None, 8)                 72        
                                                                 
 dense_5 (Dense)             (None, 3)                 27        
                                                                 
Total params: 139 (556.00 Byte)
Trainable params: 139 (556.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________



1. **Model Type:**
   - The model is a Sequential model. In a Sequential model, layers are added one at a time, and each layer has exactly one input tensor and one output tensor.

2. **Layers:**
   - **Dense Layer (Hidden Layer 1):**
     - Type: Dense
     - Output Shape: (None, 8)
     - Number of Parameters: 40
     - Activation Function: Default activation (often ReLU if not specified)
     - Description: This is the first hidden layer with 8 neurons. Each neuron is fully connected to the input, and the output shape is (None, 8). The layer has 40 parameters (weights and biases).

   - **Dense Layer (Hidden Layer 2):**
     - Type: Dense
     - Output Shape: (None, 8)
     - Number of Parameters: 72
     - Activation Function: Default activation
     - Description: This is the second hidden layer with 8 neurons. It takes the output from the first hidden layer as input and produces an output shape of (None, 8). The layer has 72 parameters.

   - **Dense Layer (Output Layer):**
     - Type: Dense
     - Output Shape: (None, 3)
     - Number of Parameters: 27
     - Activation Function: Default activation
     - Description: This is the output layer with 3 neurons, representing the number of classes in a classification task. The output shape is (None, 3), and it has 27 parameters.

3. **Total Parameters:**
   - The model has a total of 139 parameters (weights and biases). Parameters are the variables the model learns from the training data.

4. **Trainable Parameters:**
   - All 139 parameters are trainable. During the training process, the values of these parameters are updated to minimize the loss function.

5. **Non-trainable Parameters:**
   - There are no non-trainable parameters in this model. Non-trainable parameters are typically associated with layers like BatchNormalization, which have parameters that are not updated during training.



In [ ]:
# TODO: train the neural network.

# Train the model on the training data
history = model.fit(X_train, y_train_one_hot, epochs=50, validation_split=0.2)


Epoch 1/50
3/3 [==============================] - 3s 157ms/step - loss: 1.0279 - accuracy: 0.6250 - val_loss: 1.0074 - val_accuracy: 0.7500
Epoch 2/50
3/3 [==============================] - 0s 25ms/step - loss: 1.0202 - accuracy: 0.6458 - val_loss: 0.9999 - val_accuracy: 0.7500
Epoch 3/50
3/3 [==============================] - 0s 34ms/step - loss: 1.0135 - accuracy: 0.6458 - val_loss: 0.9923 - val_accuracy: 0.7500
Epoch 4/50
3/3 [==============================] - 0s 79ms/step - loss: 1.0062 - accuracy: 0.6667 - val_loss: 0.9848 - val_accuracy: 0.7917
Epoch 5/50
3/3 [==============================] - 0s 31ms/step - loss: 0.9989 - accuracy: 0.6875 - val_loss: 0.9772 - val_accuracy: 0.7917
Epoch 6/50
3/3 [==============================] - 0s 49ms/step - loss: 0.9920 - accuracy: 0.7188 - val_loss: 0.9691 - val_accuracy: 0.7917
Epoch 7/50
3/3 [==============================] - 0s 35ms/step - loss: 0.9848 - accuracy: 0.7708 - val_loss: 0.9608 - val_accuracy: 0.7917
Epoch 8/50
3/3 [==========

In [ ]:
# TODO: test the neural network and print the result on screen.

test_loss, test_accuracy = model.evaluate(X_test, y_test_one_hot)
print(f"Test accuracy: {test_accuracy}")


1/1 [==============================] - 0s 28ms/step - loss: 0.2733 - accuracy: 0.9667
Test accuracy: 0.9666666388511658


### Exercise 2: Computing a vanilla saliency map (1 points)

> Before starting, I suggest you read [1] as a warm-up. This is one of the first papers that tried to apply this kind of techniques to modern neural networks.

What do we mean by explainability? Consider the neural network $f(\cdot)$ you just trained, and a prediction $\hat{y} = f(x)$ we want to analyze. **Feature attribution** methods try to assign a weight $w_i$ to each input feature $x_i$, to understand which parts of the input have contributed the most to the explanation.

The simplest feature attribution technique, called **vanilla saliency map**, simply computes the gradient at that point:

$$
  S(x) = \left\lvert \frac{\partial f_c(x)}{\partial x} \right\rvert
$$

where  $c$ is the index corresponding to the predicted class.

✅ **Completion requirement**: Take any point from your test dataset, and compute a saliency map using `tf.GradientTape`. Check the weight to see if you can find anything to "interpret". **Note**: I am not evaluating how nice / good the explanation is, only the code.

In [ ]:
# TODO: Take an element from your test set and compute the saliency map


import tensorflow as tf
import numpy as np

# Choose a sample from the test set
test_sample = X_test[0]
test_sample = np.expand_dims(test_sample, axis=0)  # Expand dimensions to match model input

# Cast the test sample to a TensorFlow tensor
test_sample_tensor = tf.convert_to_tensor(test_sample, dtype=tf.float32)

# Watch the input pixels to compute gradient with respect to
with tf.GradientTape() as tape:
    tape.watch(test_sample_tensor)

    # Compute the model's prediction for this sample
    prediction = model(test_sample_tensor)
    predicted_class_idx = tf.argmax(prediction[0]).numpy()

    # Get the output corresponding to the predicted class
    predicted_class_output = prediction[:, predicted_class_idx]

# Compute the gradient of the output with respect to the input
saliency_map = tape.gradient(predicted_class_output, test_sample_tensor).numpy()


print("Saliency Map for the chosen sample:")
print(saliency_map)


Saliency Map for the chosen sample:
[[ 0.04512196 -0.05686454  0.06894968 -0.01811148]]


In [ ]:
# TODO: Check the saliency map to analyze the result. What can you say about the map?


The saliency map is a one-dimensional array with four values, each corresponding to the gradient of the output with respect to each of the four features of the dataset.

- The first value corresponds to the gradient with respect to the first feature. A negative value indicates that an increase in this feature would decrease the model's output for the predicted class, meaning this feature contributes inversely to the model's decision for the specific class.

- The second value is negative and larger in magnitude than the first, suggesting this feature has a stronger inverse relationship with the model's prediction for the chosen class.

- The third value is positive, indicating that increasing this feature would increase the output for the predicted class. This feature positively influences the model's decision for the class it predicted.

- The fourth value again shows a negative gradient, indicating an inverse relationship with the predicted class output.

Overall, the saliency map suggests that the second feature has the strongest inverse effect on the model's decision for the predicted class. The third feature appears to be the most positively influential. However, since the values are quite small, it implies that the changes in the input features would result in relatively small changes in the output prediction for the class.

### Exercise 3: Advanced saliency maps (1 point)

> For this exercise, you can read [2] for an overview on the limits of vanilla saliency maps and a description of SmoothGrad.

Saliency maps have several issues: most notably, they suffer from noise and they are not stable to small changes in the input or in the model (try running again the training and interpreting the same point). Many methods have been proposed to overcome this.

**[SmoothGrad](https://arxiv.org/abs/1706.03825)**, for example, computes multiple saliency maps from noisy versions of the input:

$$
  \text{SmoothGrad}(x) = \frac{1}{n}\sum_{i=1}^n S(x + \varepsilon_i), \;\; \varepsilon_i \sim \mathcal{N}(0, \sigma^2I)
$$

where $\varepsilon$ is a vector of the same shape as $x$, whose values are sampled from a normal distribution with zero mean and small variance.

🟩 **Completion requirement**: Implement the SmoothGrad procedure for the same point. Has the explanation improved? Bonus points if you can avoid running a for-loop, and by calling the gradient operation a single time.


In [ ]:
# TODO: Take an element from your test set and compute SmoothGrad. Check the results and compare with respect to the previous exercise.


sample_point = X_test[np.random.choice(len(X_test))]

n = 50  # The number of samples for SmoothGrad
sigma = 0.1  # The standard deviation for the noise

# Create a batch of noise samples
noise = np.random.normal(0, sigma, (n, sample_point.shape[0]))

# Add the noise to the sample point and create a batch of noisy samples
noisy_samples = sample_point + noise


# Convert the numpy array to a TensorFlow tensor
noisy_samples_tensor = tf.convert_to_tensor(noisy_samples, dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(noisy_samples_tensor)

    # Predict the classes of the noisy samples
    predictions = model(noisy_samples_tensor)

    # Take the class with the highest probability from the original prediction
    # for the sample point
    predicted_class = tf.argmax(model(tf.expand_dims(sample_point, 0)), axis=1).numpy()[0]

    # Gather the logits for the predicted class
    logits_for_class = predictions[:, predicted_class]

# Compute the gradients
gradients = tape.gradient(logits_for_class, noisy_samples_tensor)

# Compute the average of the gradients to get the SmoothGrad
smoothgrad = tf.reduce_mean(gradients, axis=0)

print(smoothgrad)


tf.Tensor([ 0.09972796 -0.04282816 -0.14913218  0.34133148], shape=(4,), dtype=float32)


**Check the results and compare with with respect to the previous:**

The magnitude of the gradients in the SmoothGrad saliency map seems to be generally larger. This can be a result of the noise reduction that is part of the SmoothGrad technique, which tends to highlight the areas of the input that consistently have a strong influence on the model's output.
The sign of the gradients has changed for some of the features. For example, the third feature had a positive gradient in the original map but has a negative one in the SmoothGrad map. This may indicate that the influence of this feature on the model's decision is more complex and the original saliency map might have been affected by noise.
The SmoothGrad method provide a clearer picture of feature importance by averaging the gradients over many noisy instances of the input. Therefore, we would expect the SmoothGrad map to provide a more reliable indication of which features are actually important for the model's prediction.
The last feature shows a significantly larger positive gradient in the SmoothGrad saliency map compared to the original. This suggests that, once the noise is reduced, this feature may have a stronger positive influence on the model's decision than initially indicated.


### Exercise 4: Global explanations (2 points)

The previous exercises are examples of **local** explanations, where we try to interpret a single prediction of the network. Sometimes we are interested in **global** explanations, that try to find common patterns of behaviour. Suppose we have a dataset $\mathcal{T} = \left\{x_i\right\}$ of examples, we can compute some approximate global measure of influence by averaging their saliency:

$$
\text{GlobalSaliency} = \frac{1}{n} \sum_i S(x_i)
$$

To make this exercise more interesting, we will split it into 3 parts.

**Exercise 4.1**: write a function to compute in parallel the saliency for multiple examples. Note that the resulting matrix $S$ will have shape $(n, d)$, where $n$ is the number of examples and $d$ the size of the input, which is the Jacobian of the network. Try to write the function by avoiding for-loops and multiple tapes, using the [proper tools from TensorFlow](https://www.tensorflow.org/guide/advanced_autodiff).

In [ ]:
# TODO: Write the required function, possibly avoding for-loops.
def compute_saliency(model, data):
    with tf.GradientTape() as tape:
        tape.watch(data)
        predictions = model(data)
    gradients = tape.gradient(predictions, data)
    saliency = tf.abs(gradients)
    return saliency



data = tf.convert_to_tensor(X_test, dtype=tf.float32)
compute_saliency(model, data)

<tf.Tensor: shape=(30, 4), dtype=float32, numpy=
array([[3.6744876e-08, 5.4710096e-08, 6.7799533e-08, 3.1568206e-09],
       [1.7571600e-09, 3.1856364e-09, 1.2546431e-10, 4.4862078e-09],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [3.5356096e-10, 4.4869930e-09, 1.7738390e-09, 3.1316574e-09],
       [4.4176209e-09, 2.9462106e-08, 4.6605209e-08, 2.3678489e-08],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.8172318e-10, 3.0022889e-09, 3.2279945e-09, 3.9222621e-09],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0

**Exercise 4.2**: write a function to compute the global saliency and try to explain the results.

In [ ]:
# TODO: Write the required function.
def compute_global_saliency(model, data):
    # Compute the saliency for each example
    saliency = compute_saliency(model, data)

    # Compute the global saliency by averaging across all examples
    global_saliency = tf.reduce_mean(saliency, axis=0)
    return global_saliency

data = tf.convert_to_tensor(X_test, dtype=tf.float32)
compute_global_saliency(model, data)


<tf.Tensor: shape=(4,), dtype=float32, numpy=
array([4.5043054e-09, 9.0234176e-09, 1.0457321e-08, 2.8027283e-09],
      dtype=float32)>

Each element in the array represents the average saliency value for the corresponding feature across all examples.
The values are very small (on the order of 1e-09), indicating that, on average, each feature has a relatively low impact on the model's predictions across the test examples

**Exercise 4.3**: note that a linear model $f(x) = w^\top x + b$ is an example of an *intrinsically* interpretable  model, since the weights $w$ can be checked to analyze the global saliency of each feature (see [3]).

In [ ]:
# TODO: Compare the results you obtained before with a simpler linear model.
def compare_saliency_with_linear_model(complex_model_saliency, linear_model_weights):
    # The saliency for a linear model is the absolute value of its weights
    linear_model_saliency = tf.abs(linear_model_weights)

    # Compare the two saliencies
    print("Complex Model Saliency:\n", complex_model_saliency)
    print("Linear Model Saliency:\n", linear_model_saliency)

    difference = tf.reduce_mean(tf.abs(complex_model_saliency - linear_model_saliency))
    return difference




In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)


complex_model_saliency = compute_global_saliency(model, data)

# Obtain the weights
linear_model_weights = linear_model.coef_
linear_model_weights = tf.convert_to_tensor(linear_model_weights, dtype=tf.float32)

# Compure the differences
difference = compare_saliency_with_linear_model(complex_model_saliency, linear_model_weights)

print("Difference in Saliency between Complex Model and Linear Model:", difference.numpy())

Complex Model Saliency:
 tf.Tensor([4.5043054e-09 9.0234176e-09 1.0457321e-08 2.8027283e-09], shape=(4,), dtype=float32)
Linear Model Saliency:
 tf.Tensor([0.09543703 0.02673551 0.44483158 0.41023025], shape=(4,), dtype=float32)
Difference in Saliency between Complex Model and Linear Model: 0.24430859


### Optional exercises and parting words

> ⚠ Explainability is a complex topic, with multiple issues arising from the over-abundance of techniques, their instability, etc. While an interesting research topic to pursue, never use blindly these techniques in high-stake applications!

These exercises were just a brief and short introduction to the topic of explainability. Below you can find some additional exercises to tackle if you are interested. Remember that these are not part of your grade, but I am happy to provide feedback if they are of interest to you.

1. There are dozens of possible variations on feature attribution methods, which may or may not provide better results (see [4] for a benchmarking and this nice [Distill blog post](https://distill.pub/2020/attribution-baselines/)). **[Integrated Gradients](https://arxiv.org/abs/1703.01365)** are an interesting example, where the saliency is integrated over a path ranging from an empty input to the true input. Try implementing integrated gradients.
2. **Data attribution** methods are a different class of explanation methods, which try to predict what points in the dataset where most influential to a given prediction (e.g., a picture of a cat will be especially influential on similar pictures). One example of such methods is TracIn [5], which stores checkpoints of the model during training and evaluates the correlation of the gradients. Try to implement TracIn or any other metric of data influence.
3. A recent line of research tries to use large language models (e.g., ChatGPT) to explain other models (e.g., see [Language models can explain neurons in language models](https://openai.com/research/language-models-can-explain-neurons-in-language-models)). If you have access to an LLM, you can try it! Take a specific neuron in the model, and collect the activation for multiple examples. Provide these activations to the LLM, and prompt it to provide a human-understandable explanation. What is the result?

### Final checklist

1. Carefully check all code. Insert comments when needed. Search for "TODO" to see if you forgot something.
2. Run everything one final time. *Please do not send me notebooks with errors or cells that are not working.*
3. Upload the completed notebook **before 10/11/2023 23:59** on the Google Classrom page.

### Bibliography

[1] Simonyan, K., Vedaldi, A. and Zisserman, A., 2013. [Deep inside convolutional networks: Visualising image classification models and saliency maps](https://arxiv.org/abs/1312.6034). arXiv preprint arXiv:1312.6034.

[2] Smilkov, D., Thorat, N., Kim, B., Viégas, F. and Wattenberg, M., 2017. [SmoothGrad: removing noise by adding noise](https://arxiv.org/abs/1706.03825). arXiv preprint arXiv:1706.03825.

[3] Rudin, C., 2019. [Stop explaining black box machine learning models for high stakes decisions and use interpretable models instead](https://www.nature.com/articles/s42256-019-0048-x). Nature Machine Intelligence, 1(5), pp. 206-215.

[4] Nguyen, G., Kim, D. and Nguyen, A., 2021. [The effectiveness of feature attribution methods and its correlation with automatic evaluation scores](https://proceedings.neurips.cc/paper/2021/hash/de043a5e421240eb846da8effe472ff1-Abstract.html). Advances in Neural Information Processing Systems, 34, pp.26422-26436.

[5] Pruthi, G., Liu, F., Kale, S. and Sundararajan, M., 2020. [Estimating training data influence by tracing gradient descent](https://proceedings.neurips.cc/paper/2020/hash/e6385d39ec9394f2f3a354d9d2b88eec-Abstract.html). Advances in Neural Information Processing Systems, 33, pp. 19920-19930.